In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import netCDF4 as nc
import os
import urllib
import requests
import time
import math
import shutil

In [ ]:
# Function to test whether the file is downloading or not before procedinh
def download_test(name, year, M, D, H, file):
    # Initialize a few counter/temp variables
    n = 0
    x = 0
    percent = 0
    y = 0
    z = 0
    inp = ' '
    
    # Run this loop until it is broken under specific circumstances
    while True:
        # Attempt to run main function but prepare for errors
        try:
            # Parse the name of the file in the directory to its opendap URL
            url = 'https://opendap' + name[4:19] + 'opendap' + name[23:]
            # Gather info from the URL to find the file's size
            info = requests.head(url)

            # Perform the folloeing while the file in the directory is smaller than the URL file size
            while os.path.getsize(name) != int(info.headers['Content-Length']):
                
                # Define the percent of the way downloaded the file is (diretory file size divided by URL file size)
                percent = round(os.path.getsize(name)*100/int(info.headers['Content-Length']))
                reprint()
                
                # Print the percentage the download is at
                print(year, M, D, H,'- Downloading:',str(percent)+'%',end="\r")

                # If the percent has remained unchanged for 30 seconds, stop process and wait for command
                if x == percent and n == 30:
                    # Input to continue process or save a temporary file
                    inp = input('Application paused. File is not downloading. Press Enter to continue.')
                    # Reset counter variable
                    n = 0
                    # Save temp file as to not lose progress
                    if inp.lower() == 'save':
                        temp_save(file)
                        break

                # If percent has not changed and 30 seconds have not passed, increase counter
                elif x == percent:
                    n += 1

                # Set temp variable to percent downloaded to track whether the download has stopped
                x = percent

                # Wait 1 second between iterations
                time.sleep(1)
                
            # In the case that no storage remains, stop and wait for user input
            if shutil.disk_usage(os.path.expanduser('~')).free < 70000000:
                # Continue only if a temp file has not yet been saved
                if inp.lower() != 'save':
                    # Prompt to continue or to save temp file
                    inp = input('Disk is out of space. Clear issue then hit Enter to continue.')
                    if inp.lower() == 'save':
                        temp_save(file)
                        break

            break

        # Do not stop program in the case a file cannot be found
        except FileNotFoundError:
            reprint()
            print('File not found:',name,end='\r')
            # Wait 15 seconds between iterations and check 4 times before accepting the file will not be downloaded and moving on
            time.sleep(15)
            z += 1
            if z >= 4:
                break

        # Do not stop the program in the case of a connection error
        except requests.exceptions.ConnectionError:
            # Increase count everytime connection is not achieved
            y += 1
            
            # Prompt user intervention of 5 minutes have passed without connection
            if y == 20:
                inp = input('Program lost connection. Press Enter to continue.')
                y = 0
                if inp.lower() == 'save':
                    temp_save(file)
                    break
                    
            # Pause for 15 seconds before retrying loop
            else:
                reprint()
                print('Retrying connection.', end = '\r')
                time.sleep(15)
            
        # Do not stop program in the case that the URL file size cannot be sourced
        except KeyError:
            reprint()
            print(year, M, D, H,'Content length could not be found.')
            try: 
                while True:
                    reprint()
                    print(year, M, D, H,'- Downloading',end="\r")
                    
                    # Only allow the program to continue if the file size has stopped changing and is greater than 40MB
                    if os.path.getsize(name) == x and os.path.getsize(name) > 40000000:
                        break
                        
                    # Prompt user intervention if the file has stopped growing but is less than 40MB
                    elif os.path.getsize(name) == x:
                        inp = input('File size is small. Check to ensure this is correct and hit Enter to continue.')
                        if inp.lower() == 'save':
                            temp_save(file)
                            break
                            
                    # If file is still growing, check size again and wait 5 seconds
                    else:
                        x = os.path.getsize(name)
                        time.sleep(5)
                        
                    if inp.lower() == 'save':
                        break
                
            # If the program cannot find the desired file
            except FileNotFoundError:
                reprint()
                print('File not found:',name,end='\r')
                # Wait 15 seconds between iterations and check 4 times before accepting the file will not be downloaded and moving on
                time.sleep(15)
                z += 1
                if z >= 4:
                    break
                    
                    
            break
                    
# A function that simply clears the command line
def reprint():
    print(' '*500,end='\r')
    
# A function that saves all months that have been fully downloaded
def temp_save(file):
    file = file.drop(labels = 'VZA',axis = 1)
    file = xr.Dataset.from_dataframe(file)
    if year+'_TEMP.nc' in os.listdir():
        file = xr.merge([xr.open_dataset(year+'_TEMP.nc'),file])
        os.remove(year+'_TEMP.nc')
    file.to_netcdf(year+'_TEMP.nc')
    input('Temp file saved, please end program.')
        
        

In [ ]:
#In Terminal
#cd ~
#USERNAME_ = <your username>
#PASSWORD = <your password>
#touch .netrc
#echo "machine urs.earthdata.nasa.gov login $USERNAME_ password $PASSWORD" > .netrc
#nano .netrc (check to ensure your password and username are correct)
#chmod 0600 .netrc
#touch .urs_cookies
#navigate to your desired download folder: cd <path>
#cat .netrc
#Test by downloading a single file: wget --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --auth-no-challenge=on --keep-session-cookies --content-disposition <URL>
#Download all files: wget --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --auth-no-challenge=on --keep-session-cookies --content-disposition --recursive --wait=.05 --no-parent --reject "index.html*" --execute robots=off -c <URL>




# Mute Pandas warnings
pd.options.mode.chained_assignment = None

# Prompt download year to use later
year = input('What year would you like to download?')

# Prompt start month just in case a temp file has been created
month_s = input('What month would you like to start with? Enter numerical value.')
month_s = int(month_s)

VA = input('What viewing zenith angle would you like?')
VA = float(VA)

# Define parts of the path the files will be downloaded to (URL is a legacy name)
url1 = 'asdc.larc.nasa.gov/data/CERES/SSF/NPP-FM5-VIIRS_Edition2A/' + year + '/'
url2 = '/CER_SSF_NPP-FM5-VIIRS_Edition2A_200203.' + year

# Initialize all variables
skip_count = 0

total = []

time_ = np.array([])
lon = np.array([])
lat = np.array([])
SWrad = np.array([])
LWrad = np.array([])
sza = np.array([])
vza = np.array([])
TOAincoming = np.array([])

sw_day = np.array([])
lat_day = np.array([])
lon_day = np.array([])
time_day = np.array([])
sza_day = np.array([])

# Loop through all desired months, every day, and every hour
for M in range(month_s,13):
    M =str(M)
    for D in range(1,32):
        D=str(D)
        for H in range(24):
            H=str(H)
            while len(D)<2:
                D = '0'+D
            while len(H)<2:
                H = '0'+H
            while len(M)<2:
                M = '0'+M
            
            # Do nothing in the case the month-day combination does not exist
            if ((int(M)==2 and int(D)>28) and (int(year)%4 != 0)) or ((int(M)==2) and (int(D)>29)) or (((int(M)==4) or (int(M)==6) or (int(M)==9) or (int(M)==11)) and (int(D)>30)):
                None
            else:
                # Check if the file is being downloaded before moving on
                download_test(url1+M+url2+M+D+H+'.nc', year, M, D, H, total)
                
                # Attemp to open file and parse data
                try:
                    # Open file associated with current M-D-H combinations
                    file = nc.Dataset(url1+M+url2+M+D+H+'.nc')
                    
                    # Retrieve desired variables
                    time_ = np.append(time_,np.array(file.groups['Time_and_Position'].variables['julian_observation_time'][:]))
                    lon = np.append(lon,np.array(file.groups['Time_and_Position'].variables['instrument_fov_longitude'][:]))
                    lat = np.append(lat,np.array(file.groups['Time_and_Position'].variables['instrument_fov_latitude'][:]))
                    SWrad = np.append(SWrad,np.array(file.groups['Unfiltered_Radiances'].variables['shortwave_radiance'][:]))
                    LWrad = np.append(LWrad,np.array(file.groups['Unfiltered_Radiances'].variables['longwave_radiance'][:]))
                    sza = np.append(sza,np.array(file.groups['Viewing_Angles'].variables['solar_zenith_angle'][:]))
                    vza = np.append(vza,np.array(file.groups['Viewing_Angles'].variables['view_zenith_angle'][:]))
                    TOAincoming = np.append(TOAincoming,np.array(file.groups['TOA_and_Surface_Fluxes'].variables['toa_incoming_solar_radiation'][:]))
                    
                    # Delete file variabel for storage and delete actual file for storage
                    file.close()
                    del file
                    os.remove(url1+M+url2+M+D+H+'.nc')
                    
                    # Parse all variables to viewing zenith angle less than .15
                    time_ = time_[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    lon = lon[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    lat = lat[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    SWrad = SWrad[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    LWrad = LWrad[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    sza = sza[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    TOAincoming = TOAincoming[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    vza = vza[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    
                    # Print the M-D-H combo of the file most recently completed
                    reprint()
                    print(year,M,D,H)
                    # Reset skip counter due to successful download
                    skip_count = 0
                    
                # Do not stop program if file does not exist
                except FileNotFoundError:
                    # Wait 5 seconds and try previous process again
                    time.sleep(5)
                    try:
                        reprint()
                        print('Cannot find', year, M, D, H+'.',end='\r')
                        download_test(url1+M+url2+M+D+H+'.nc',year,M,D,H,total)
                        
                        file = nc.Dataset(url1+M+url2+M+D+H+'.nc')
                        
                        time_ = np.append(time_,np.array(file.groups['Time_and_Position'].variables['julian_observation_time'][:]))
                        lon = np.append(lon,np.array(file.groups['Time_and_Position'].variables['instrument_fov_longitude'][:]))
                        lat = np.append(lat,np.array(file.groups['Time_and_Position'].variables['instrument_fov_latitude'][:]))
                        SWrad = np.append(SWrad,np.array(file.groups['Unfiltered_Radiances'].variables['shortwave_radiance'][:]))
                        LWrad = np.append(LWrad,np.array(file.groups['Unfiltered_Radiances'].variables['longwave_radiance'][:]))
                        sza = np.append(sza,np.array(file.groups['Viewing_Angles'].variables['solar_zenith_angle'][:]))
                        vza = np.append(vza,np.array(file.groups['Viewing_Angles'].variables['view_zenith_angle'][:]))
                        TOAincoming = np.append(TOAincoming,np.array(file.groups['TOA_and_Surface_Fluxes'].variables['toa_incoming_solar_radiation'][:]))
                    
                        file.close()
                        del file
                        os.remove(url1+M+url2+M+D+H+'.nc')
                        
                        time_ = time_[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                        lon = lon[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                        lat = lat[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                        SWrad = SWrad[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                        LWrad = LWrad[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                        sza = sza[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                        TOAincoming = TOAincoming[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                        vza = vza[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                        
                    # If file still does not exist, simply skip it
                    except FileNotFoundError:
                        reprint()
                        print('Skipping', year,M,D,H+'.')
                        # Determine how many files have been skipped in a row
                        if skip_count >= 5:
                            # If 5 files have been skippes, call for user interaction and allow the chance to save progress
                            skip_count = 0
                            inp = input('Five files have been skipped in a row. Ensure this is correct then hit Enter.')
                            if inp.lower() == 'save':
                                temp_save(total)
                        # If less than 5 files have been skipped, add to the counter
                        else:
                            skip_count += 1
                        
                # In the case that a downloaded file cannot be opened, reattempt
                except OSError:
                    reprint()
                    print('Reattempting', year,M,D,H+'.')
                    
                    # Delete current iteration of file download
                    os.remove(url1+M+url2+M+D+H+'.nc')
                    
                    # Define opendap URL to donwload from
                    url = 'https://opendap.larc.nasa.gov/opendap/CERES/SSF/NPP-FM5-VIIRS_Edition2A/'+year+'/'+M+url2+M+D+H+'.nc'
                    
                    # Download file
                    urllib.request.urlretrieve(url,'/Users/atocreu/Desktop/REU/'+year+M+D+H+'.nc')
                    
                    # Begin normal process
                    file = nc.Dataset(year+M+D+H+'.nc')
                    
                    time_ = np.append(time_,np.array(file.groups['Time_and_Position'].variables['julian_observation_time'][:]))
                    lon = np.append(lon,np.array(file.groups['Time_and_Position'].variables['instrument_fov_longitude'][:]))
                    lat = np.append(lat,np.array(file.groups['Time_and_Position'].variables['instrument_fov_latitude'][:]))
                    SWrad = np.append(SWrad,np.array(file.groups['Unfiltered_Radiances'].variables['shortwave_radiance'][:]))
                    LWrad = np.append(LWrad,np.array(file.groups['Unfiltered_Radiances'].variables['longwave_radiance'][:]))
                    sza = np.append(sza,np.array(file.groups['Viewing_Angles'].variables['solar_zenith_angle'][:]))
                    vza = np.append(vza,np.array(file.groups['Viewing_Angles'].variables['view_zenith_angle'][:]))
                    TOAincoming = np.append(TOAincoming,np.array(file.groups['TOA_and_Surface_Fluxes'].variables['toa_incoming_solar_radiation'][:]))
                    
                    file.close()
                    del file
                    os.remove(year+M+D+H+'.nc')
                        
                    time_ = time_[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    lon = lon[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    lat = lat[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    SWrad = SWrad[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    LWrad = LWrad[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    sza = sza[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    TOAincoming = TOAincoming[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    vza = vza[np.where((vza<(VA+.15))&(vza>(VA-.15)))]
                    
                    print(year,M,D,H,'reattempt successful.')
                    
                    
    # At the end of the month announce that the month has finished and must be organized
    print('Compiling data for',year,M)

    # Add all retrieved variables to a pandas data fram
    data = pd.DataFrame(data={
        'time':time_,
        'lon':lon,
        'lat':lat,
        'SWrad':SWrad,
        'LWrad':LWrad,
        'SZA':sza,
        'VZA':vza,
        'TOA_in':TOAincoming
    })
    
    # Clear all variables for next month
    time_ = np.array([])
    lon = np.array([])
    lat = np.array([])
    SWrad = np.array([])
    LWrad = np.array([])
    sza = np.array([])
    vza = np.array([])
    TOAincoming = np.array([])
    
    # Set lon values to correct orientation, turn Julian datetime into normal time, replace bad values with NaN
    data.lon = (data.lon+180)%360 - 180
    data.time = pd.to_datetime(data.time,origin='julian',unit='D')
    data.LWrad[data.LWrad > 2e+38] = np.NaN   
    data.SWrad[data.SWrad > 2e+38] = np.NaN     
    data.TOA_in[data.TOA_in > 2e+38] = 0
    
    # Iterate through all indexes in new dataframe
    for i in range(len(data.time)):
        # When i is divisible by 100, print the percent of the way through the indexing the loop is
        if i % 100 == 0:
            reprint()
            print(str(round(100*i/len(data.time)))+'%',end="\r")

        # Only acvept values that are not NAN. Also, round to the nearest lat and lon for averaging
        if not (math.isnan(data.lat.iloc[i]) or math.isnan(data.lon.iloc[i])):
            data.lon[i] = round(data.lon.iloc[i])
            data.lat[i] = round(data.lat.iloc[i])

        # Attempt to set the time index to stop at the day variable
        try:
            data.time[i] = datetime.datetime(data.time[i].year,data.time[i].month,data.time[i].day)
        # If the value is not a time (NaT), simply use the previous time
        except TypeError:
            data.time[i] = datetime.datetime(data.time[i-1].year,data.time[i-1].month,data.time[i-1].day)
            
    # Gather variables for only the day time
    sw_day = data.SWrad[data.SZA < 90]
    lat_day = data.lat[data.SZA < 90]
    lon_day = data.lon[data.SZA < 90]
    time_day = data.time[data.SZA < 90]
    sza_day = data.SZA[data.SZA < 90]
    
    
    # Append these day variables to a new Pandas dataframe        
    data_day = pd.DataFrame(data={
        'time':time_day,
        'lon':lon_day,
        'lat':lat_day,
        'SWrad_day':sw_day,
        'SZA_day':sza_day
    })
    
    # Clear the variables
    sw_day = np.array([])
    lat_day = np.array([])
    lon_day = np.array([])
    time_day = np.array([])
    sza_day = np.array([])


    # Turn both dataframes into multi-index dataframes with time, lat, lon being the indices
    # Average all points of data that have the same three indices
    
    data.set_index(['time','lat','lon'],inplace=True)

    data = data.groupby(level=data.index.names).mean()
    
    data_day.set_index(['time','lat','lon'],inplace=True)

    data_day = data_day.groupby(level=data_day.index.names).mean()
    
    # Join the daytime values to the main dataframe and then delete it
    data = data.join(data_day)
    
    del data_day
    
    # If the current month iteration is equal to the starting month, call it the dataframe total so that it may be appended to
    if int(M) == month_s:
        total = data
        
    # If the current month is not the starting month, append this months data to total
    else:
        total = pd.concat([total,data])
        
    # del data so that it may be reused
    del data
    

    
# Delete the viewing zenith angle column because it is no longer needed
total = total.drop(labels = 'VZA',axis = 1)
# Turn the dataframe into an xarray dataset because it is better suited to multiple coordinates
total = xr.Dataset.from_dataframe(total)
# Test if temp file was saved and combine into year long file if it was
if year+'_TEMP.nc' in os.listdir():
    total = xr.merge([xr.open_dataset(year+'_TEMP.nc'),total])
    os.remove(year+'_TEMP.nc')
# Save the dataset to a netcdf file
total.to_netcdf(year+'_VZA_'+str(VA)+'_ERB.nc')

reprint()
print('DONE')